In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import lit

In [0]:
class Refine:
    def __init__(self, df: DataFrame):
        self.df = df

    def renamecol(self, colmap):
        self.df = self.df.toDF(*colmap)
        return self
    
    def addcol(self, colname, datatype, colvalue=None):
        self.df = self.df.withColumn(colname, lit(colvalue).cast(datatype))
        return self
    
    def load(self):
        return self.df